In [5]:
import networkx as nx
from collections import defaultdict
import heapq

In [7]:
def load_graph(file_path):
    """
    Load the Facebook dataset and create an undirected graph
    """
    G = nx.Graph()
    with open(file_path, 'r') as f:
        for line in f:
            u, v = map(int, line.strip().split())
            G.add_edge(u, v)
    return G


In [8]:
def betweenness_centrality(G):
    """
    Calculate the intermediate centrality
    """
    betweenness = defaultdict(float)
    nodes = list(G.nodes())
    n = len(nodes)
    
    for s in nodes:
        stack = []
        pred = defaultdict(list)
        sigma = defaultdict(lambda: 0)
        dist = defaultdict(lambda: -1)
        sigma[s] = 1
        dist[s] = 0
        queue = [s]
        
        # BFS
        while queue:
            v = queue.pop(0)
            stack.append(v)
            for w in G.neighbors(v):
                if dist[w] < 0:
                    queue.append(w)
                    dist[w] = dist[v] + 1
                if dist[w] == dist[v] + 1:
                    sigma[w] += sigma[v]
                    pred[w].append(v)
        
        # accumulate
        delta = defaultdict(float)
        while stack:
            w = stack.pop()
            for v in pred[w]:
                delta[v] += (sigma[v] / sigma[w]) * (1 + delta[w])
            if w != s:
                betweenness[w] += delta[w]
    
    # normalization
    for v in betweenness:
        betweenness[v] /= ((n - 1) * (n - 2) / 2)
    
    return betweenness

def get_top_10_nodes(betweenness):
    """
    Returns the first 10 nodes
    """
    return [node for node, _ in heapq.nlargest(10, betweenness.items(), key=lambda x: x[1])]

In [9]:
def pagerank_centrality(G, alpha=0.85, beta=0.15, epsilon=1e-6, max_iter=100):
    """
    Calculate PageRank centrality
    """
    if G is None:
        return {}
    N = G.number_of_nodes()
    if N == 0:
        return {}
    
    # Initialize
    pr = {node: 1.0 / N for node in G.nodes()}
    pr_new = defaultdict(float)
    
    # Iterate until convergence or the maximum number of iterations is reached
    for iteration in range(max_iter):
        total_diff = 0.0
        # New score
        for v in G.nodes():
            pr_new[v] = beta  
            # Add up the neighbors' contributions
            for u in G.neighbors(v):
                pr_new[v] += alpha * pr[u] / G.degree(u)
        
        # Calculate the convergence difference
        for v in G.nodes():
            total_diff += abs(pr_new[v] - pr[v])
            pr[v] = pr_new[v]
    
    # normalization
    total_pr = sum(pr.values())
    if total_pr > 0:
        for v in pr:
            pr[v] /= total_pr
    
    return pr

In [14]:
def main():
    file_path = "3. data.txt"
    G = load_graph(file_path)
    
    betweenness = betweenness_centrality(G)
    top_10_betweenness = get_top_10_nodes(betweenness)
    pagerank_scores = pagerank_centrality(G)
    top_10_pagerank = get_top_10_nodes(pagerank_scores)
        
    
    print("Top-10 nodes by Betweenness Centrality：", " ".join(map(str, top_10_betweenness)))
    print("Top-10 nodes by PageRank Centrality：", " ".join(map(str, top_10_pagerank)))

In [16]:
if __name__ == "__main__":
    main()

Top-10 nodes by Betweenness Centrality： 107 1684 3437 1912 1085 0 698 567 58 428
Top-10 nodes by PageRank Centrality： 3437 107 1684 0 1912 348 686 3980 414 483
